In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import zipfile 
from zipfile import ZipFile 
import sklearn



# opening the zip file in READ mode 
file_name = "playground-series-s4e7.zip"
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir()
    
    
# import
df_zip = zipfile.ZipFile(file_name)
train = pd.read_csv(df_zip.open('train.csv'))
test = pd.read_csv(df_zip.open('test.csv'))
sample_submission = pd.read_csv(df_zip.open('sample_submission.csv')) 

import random

n = len(train) # Calculate number of rows in file
s = n//10  # sample size of 10%
skip = sorted(random.sample(range(1, n+1), n-s))   
train_sample = pd.read_csv(df_zip.open('train.csv'), skiprows = skip)
train_sample

File Name                                             Modified             Size
sample_submission.csv                          2024-06-24 13:46:18     99708270
test.csv                                       2024-06-24 13:46:24    433918183
train.csv                                      2024-06-24 13:46:48    662779095


,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,11,Female,25,1,10.0,0,< 1 Year,Yes,2630.0,152.0,30,0
1,32,Male,43,1,10.0,1,1-2 Year,No,18361.0,152.0,260,0
2,40,Female,67,1,28.0,0,1-2 Year,Yes,2630.0,122.0,81,0
3,52,Male,23,1,8.0,1,< 1 Year,No,24282.0,152.0,94,0
4,57,Female,28,1,3.0,1,< 1 Year,No,25461.0,160.0,242,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1150474,11504778,Female,36,1,28.0,0,1-2 Year,Yes,49276.0,124.0,293,0
1150475,11504780,Male,26,1,11.0,1,< 1 Year,No,35401.0,152.0,234,0
1150476,11504783,Male,25,1,3.0,1,< 1 Year,No,23405.0,152.0,113,0
1150477,11504784,Male,25,1,28.0,0,< 1 Year,Yes,48240.0,153.0,37,1


In [4]:
# drop id
train = train.drop('id', axis = 1)

# convert obj to cat
categorical = ["Region_Code", "Policy_Sales_Channel", "Gender", "Vehicle_Damage", "Vehicle_Age"]
train[["Region_Code", "Policy_Sales_Channel"]] = train[["Region_Code", "Policy_Sales_Channel"]].astype('int32')
train[categorical] = train[categorical].astype("category")

# train sample
train[["Region_Code", "Policy_Sales_Channel"]] = train[["Region_Code", "Policy_Sales_Channel"]].astype('int32')
train[categorical] = train[categorical].astype("category")

# convert numerics into int 32 

import numpy as np
# integer
d = dict.fromkeys(train.select_dtypes(np.int64).columns, np.int32)
train = train.astype(d)

# float
d2 = dict.fromkeys(train.select_dtypes(np.float64).columns, np.int32)
train = train.astype(d2)

train_sample = train_sample.drop('id', axis = 1)

# convert obj to cat
train_sample[["Region_Code", "Policy_Sales_Channel"]] = train_sample[["Region_Code", "Policy_Sales_Channel"]].astype('int32')
train_sample[categorical] = train_sample[categorical].astype("category")

# train_sample sample
train_sample[["Region_Code", "Policy_Sales_Channel"]] = train_sample[["Region_Code", "Policy_Sales_Channel"]].astype('int32')
train_sample[categorical] = train_sample[categorical].astype("category")

# convert numerics into int 32 

import numpy as np
# integer
d = dict.fromkeys(train_sample.select_dtypes(np.int64).columns, np.int32)
train_sample = train_sample.astype(d)

# float
d2 = dict.fromkeys(train_sample.select_dtypes(np.float64).columns, np.int32)
train_sample = train_sample.astype(d2)

In [5]:
from sklearn.model_selection import train_test_split
X = train.drop("Response", axis = 1)
y = train["Response"]


X_train, X_val, y_train, y_val = train_test_split(X,y,
                                                    test_size = 0.2,
                                                    random_state = 42,
                                                    stratify = y)

Xs = train_sample.drop("Response", axis = 1)
ys = train_sample["Response"]


Xs_train, Xs_val, ys_train, ys_val = train_test_split(Xs,ys,
                                                    test_size = 0.2,
                                                    random_state = 42,
                                                    stratify = ys)

In [6]:
from sklearn.metrics import f1_score, recall_score, confusion_matrix,roc_auc_score
import xgboost as xgb

xgstrain = xgb.DMatrix(Xs_train,ys_train, enable_categorical= True)
xgstest  = xgb.DMatrix(Xs_val,ys_val, enable_categorical= True)
xgtrain = xgb.DMatrix(X_train,y_train, enable_categorical= True)
xgtest  = xgb.DMatrix(X_val,y_val, enable_categorical= True)

params_1 = {"objective": "binary:logistic",
            "device":"cuda",
            "tree_method": "hist",
            "enable_categorical": True,
            "max_cat_to_onehot" : 1}

n = 1000


In [1]:
%%capture --no-stderr
results = xgb.cv(params_1,
                 xgstrain,
                 num_boost_round = n,
                 nfold=5,
                 metrics = ["logloss","auc","error"],
                 early_stopping_rounds=20
                 )
results


NameError: name 'xgb' is not defined

In [8]:
params_2 = {"objective": "binary:logistic",
            "device":"cuda",
            "tree_method": "hist",
            "enable_categorical": True,
            "max_cat_to_onehot" : 1}
clf_1 = xgb.XGBClassifier(**params_2)


In [9]:


clf_1.fit((Xs_train),ys_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=1, max_delta_step=None,
              max_depth=None, max_leaves=None, min_child_weight=None,
              missing=nan, monotone_constraints=None, multi_strategy=None,
              n_estimators=None, n_jobs=None, num_parallel_tree=None,
              random_state=None, ...)

In [11]:

pred_1 = clf_1.predict(Xs_val)

print(f"f1 score : {f1_score(ys_val, pred_1)}\n")
print(f"confusion Matrix:\n{confusion_matrix(ys_val, pred_1)}\n")

f1 score : 0.16020154191707642

confusion Matrix:
[[199789   2093]
 [ 25575   2639]]



In [12]:
import xgboost as xgb 
import hyperopt
from sklearn.model_selection import cross_val_score 
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval 
from hyperopt.early_stop import no_progress_loss

search_space = {
    'max_depth': hp.choice("max_depth", np.arange(1,20,1,dtype=int)),
    'eta'      : hp.uniform("eta", 0, 1),
    'gamma'    : hp.uniform("gamma", 0, 10e1),
    'reg_alpha': hp.uniform("reg_alpha", 10e-7, 10),
    'reg_lambda' : hp.uniform("reg_lambda", 0,1),
    'colsample_bytree': hp.uniform("colsample_bytree", 0.5,1),
    'colsample_bynode': hp.uniform("colsample_bynode", 0.5,1), 
    'colsample_bylevel': hp.uniform("colsample_bylevel", 0.5,1),
    'n_estimators': hp.choice("n_estimators", np.arange(100,1000,10,dtype='int')),
    'min_child_weight' : hp.choice("min_child_weight", np.arange(1,10,1,dtype='int')),
    'max_delta_step' : hp.choice("max_delta_step", np.arange(1,10,1,dtype='int')),
    'subsample' : hp.uniform("subsample",0.5,1),
    'objective' : 'binary:logistic',
    'eval_metric' : 'aucpr',
    'seed' : 44,
    'device':"cuda",
    'tree_method': "hist",
}

In [13]:
def xgb_objective(space):
  results = xgb.cv(space, 
                   dtrain=xgstrain, #DMatrix (xgboost specific)
                   num_boost_round=500, 
                   nfold=5, 
                   stratified=True,  
                   early_stopping_rounds=20,
                   metrics = ['logloss','auc','aucpr','error'])
  
  best_score = results['test-auc-mean'].max()
  return {'loss':-best_score, 'status': STATUS_OK}

In [14]:
trials = Trials()

best_hyperparams = fmin(fn=xgb_objective, space=search_space,algo=tpe.suggest,max_evals=500,trials=trials, 
                        return_argmin=False, early_stop_fn=no_progress_loss(10))

best_params = best_hyperparams.copy()

# `eval_metric` is a key that is not a hyperparameter of the classifier
if 'eval_metric' in best_params:
  best_params = {key:best_params[key] for key in best_params if key!='eval_metric'}

best_params

  0%|          | 0/500 [00:00<?, ?trial/s, best loss=?]

c:\Users\Zookeeper\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [21:26:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)

c:\Users\Zookeeper\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [21:26:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)



  0%|          | 1/500 [00:06<55:25,  6.66s/trial, best loss: -0.8657326726270353]

c:\Users\Zookeeper\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [21:26:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)



  0%|          | 2/500 [00:11<48:07,  5.80s/trial, best loss: -0.8657326726270353]

...

c:\Users\Zookeeper\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [21:27:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)



  3%|▎         | 14/500 [01:06<28:34,  3.53s/trial, best loss: -0.8717859882822238]

c:\Users\Zookeeper\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [21:27:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)

c:\Users\Zookeeper\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [21:27:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)



  3%|▎         | 15/500 [01:13<39:24,  4.87s/trial, best loss: -0.8717859882822238]


{'colsample_bylevel': 0.9126957097003363,
 'colsample_bynode': 0.540387550285983,
 'colsample_bytree': 0.9937598685147964,
 'device': 'cuda',
 'eta': 0.16868336243949966,
 'gamma': 6.1131795697817015,
 'max_delta_step': np.int64(5),
 'max_depth': np.int64(14),
 'min_child_weight': np.int64(8),
 'n_estimators': np.int64(750),
 'objective': 'binary:logistic',
 'reg_alpha': 3.403329002411318,
 'reg_lambda': 0.4815379962491406,
 'seed': 44,
 'subsample': 0.8372891534980578,
 'tree_method': 'hist'}

In [19]:
from sklearn.metrics import precision_score,accuracy_score,recall_score,f1_score
clf_2 = xgb.XGBClassifier(**best_params, enable_categorical = True)
clf_2.fit(Xs_train, ys_train)

pred_2 = clf_2.predict(Xs_val)

print(f"f1 score : {f1_score(ys_val, pred_2)}\n"
      f"accuracy score : {accuracy_score(ys_val, pred_2)}\n" 
       f"recall score : {recall_score(ys_val, pred_2)}\n"  )

f1 score : 0.11479957906821009
accuracy score : 0.8793590501355956
recall score : 0.06379811441128518



In [20]:
clf_3 = xgb.XGBClassifier(**best_params, enable_categorical = True)
clf_3.fit(X_train, y_train)
clf_3.save_model("ICS_xgb_1.json")